In [ ]:
import cv2
import numpy as np 
import pandas as pd 
import os
from keras.utils.np_utils import to_categorical 
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation,Dropout,BatchNormalization
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from glob import glob
from tensorflow.keras.applications import VGG16,EfficientNetB3
from tensorflow import keras

In [ ]:
NUMBER_CLASSES=10

def get_image(path, img_rows, img_cols, color_type=3): #get image
    img = cv2.imread(path,1)
    resized = cv2.resize(img, (img_rows, img_cols)) # Reduce size
    return resized
#------------------------------------------------train------------------------------------------------------------------------------
def load_train(img_rows, img_cols, color_type=3): #train images and train labels
    train_images = [] 
    train_labels = []
    for classed in range(NUMBER_CLASSES):
        print('Loading directory c{}'.format(classed))
        files = glob(os.path.join('../input/state-farm-distracted-driver-detection/imgs/train/c' + str(classed), '*.jpg'))
        for file in files:
            img = get_image(file, img_rows, img_cols, color_type)
            train_images.append(img)
            train_labels.append(classed)
    return train_images, train_labels 

def read_and_normalize_train_data(img_rows, img_cols, color_type):
    X, labels = load_train(img_rows, img_cols, color_type)
    enc = LabelEncoder()                        
    P = enc.fit_transform(labels)                ## convert string labels to numbers 
    y = to_categorical(P)                        ## convert number to one-hot-encode form 
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2) # split into train and test
    x_train = np.array(x_train).reshape(-1,img_rows,img_cols,color_type)
    x_test = np.array(x_test).reshape(-1,img_rows,img_cols,color_type)
    return x_train, x_test, y_train, y_test
#----------------------------------------------------------------------------------------------------------------------------------------------

#-------------------------------------------------------test------------------------------------------------------------------------------------
def load_test(size=200000, img_rows=64, img_cols=64, color_type=3):
    path = os.path.join('../input/state-farm-distracted-driver-detection/imgs/test', '*.jpg')
    files = sorted(glob(path))
    X_test, X_test_id = [], []
    total = 0
    files_size = len(files)
    for file in files:
        if total >= size or total >= files_size:
            break
        file_base = file.split(os.path.sep)[-1]
        img = get_image(file, img_rows, img_cols, color_type)
        X_test.append(img)
        X_test_id.append(file_base)
        total += 1
    return X_test, X_test_id

def read_and_normalize_sampled_test_data(size, img_rows, img_cols, color_type=3):
    test_data, test_ids = load_test(size, img_rows, img_cols, color_type)   
    test_data = np.array(test_data)
    test_data = test_data.reshape(-1,img_rows,img_cols,color_type)
    return test_data, test_ids
#-------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
test_data = pd.read_csv('/kaggle/input/state-farm-distracted-driver-detection/sample_submission.csv')
test_data.shape

In [ ]:
img_rows = 64 # dimension of images
img_cols = 64
color_type = 3
test_samples = test_data.shape[0]

# loading train images
x_train, x_test, y_train, y_test = read_and_normalize_train_data(img_rows, img_cols, color_type)


In [ ]:
# loading validation images
test_files, test_targets = read_and_normalize_sampled_test_data(test_samples, img_rows, img_cols, color_type)

In [ ]:
classes = {'c0': 'Safe driving', 
            'c1': 'Texting - right', 
            'c2': 'Talking on the phone - right', 
            'c3': 'Texting - left', 
            'c4': 'Talking on the phone - left', 
            'c5': 'Operating the radio', 
            'c6': 'Drinking', 
            'c7': 'Reaching behind', 
            'c8': 'Hair and makeup', 
            'c9': 'Talking to passenger'}

In [ ]:
def Create_model2():
    model = Sequential()

    model.add(Conv2D(32,(3,3),activation='relu',input_shape=(img_rows, img_cols, color_type)))
    model.add(Conv2D(32,(3,3),activation='relu',padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2),padding='same'))

    model.add(Conv2D(64,(3,3),activation='relu',padding='same'))
    model.add(Conv2D(64,(3,3),activation='relu',padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2),padding='same'))

    model.add(Conv2D(128,(3,3),activation='relu',padding='same'))
    model.add(Conv2D(128,(3,3),activation='relu',padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2),padding='same'))

    model.add(Flatten())
    model.add(Dense(512,activation='relu'))
    model.add(Dense(128,activation='relu'))
    model.add(Dense(128,activation='relu'))
    model.add(Dense(10,activation='softmax'))
    return model

In [ ]:
model = Create_model2()

model.summary()

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=20,batch_size=40, verbose=1)

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
score1 = model.evaluate(x_test, y_test, verbose=1)
print('Loss: ', score1[0])
print('Accuracy: ', score1[1]*100, ' %')

In [ ]:
predictions = model.predict(test_files, batch_size=40, verbose=0)

In [ ]:
for i in range(10):
    img = test_files[i]
    plt.imshow(img, cmap='gray')

    print('Y prediction: {}'.format(predictions[i]))
    print('Predicted: {}'.format(classes.get('c{}'.format(np.argmax(predictions[i])))))
    
    plt.show()

In [ ]:

result = pd.DataFrame(predictions, columns=['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9'])
result.loc[:, 'img'] = pd.Series(test_targets, index=result.index)
    
sub_file = os.path.join('submission' + '.csv')
    
result.to_csv(sub_file, index=False)

In [ ]:
test_csv = pd.read_csv('./submission.csv')
test_csv